In [1]:
import requests
import json
import pandas as pd
from pprint import pprint
from time import sleep

from spotify_credentials import client_id, client_secret

In [ ]:
#getting spotify access token
url = "https://accounts.spotify.com/api/token"
header = {"Content-Type": "application/x-www-form-urlencoded"}
data = {"grant_type":"client_credentials",
        "client_id" : client_id,
        "client_secret" : client_secret}

r = requests.post(url, data=data, headers=header).json()

spotify_api_key = r['access_token']



In [2]:
songdf = pd.read_csv(r"")
songdf["explicit"] = True

#songdf = songdf.iloc[:,[5,1,2,3,4,6,7,8,9,10]]
songdf = songdf.head(1)

In [ ]:
spotify_analysis_endpoint = "https://api.spotify.com/v1/audio-analysis/"
spotify_headers = {'Authorization': 'Bearer {}'.format(spotify_api_key)}

for index, row in songdf.iterrows():

    song_id = str(songdf.iat[index,0])

    request_url = spotify_analysis_endpoint + song_id

    try:
        spotify_response = requests.get(request_url, headers = spotify_headers).json()
        songdf.iat[index,5] = spotify_response["track"]["duration"]
        songdf.iat[index,6] = spotify_response["track"]["loudness"]
        songdf.iat[index,7] = spotify_response["track"]["tempo"]
        songdf.iat[index,8] = spotify_response["track"]["time_signature"]
        songdf.iat[index,9] = spotify_response["track"]["key"]
        sleep(0.5)
        
    except:
        print(f"Failure on ID: {song_id}")
        print(spotify_response)
        pass
        


songdf.head()